In [12]:
# Initial imports
import os
import re
import json
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas import json_normalize
from dotenv import load_dotenv
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, ConceptsOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, RelationsOptions, SemanticRolesOptions
from dotenv import load_dotenv
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
load_dotenv()

True

In [2]:
# Connect to IBM API
nlu_apikey = "insert here"
nlu_url = "insert here"

In [ ]:
# Import Dataset 
dataset = pd.read_csv('/Users/erahm/chatter/Datasets/Monkeypox.csv')
dataset.head(100)

Data Cleaning

In [ ]:
# Drop any duplicate messages
updated_df = dataset.drop_duplicates(keep='last')
updated_df

In [ ]:
# Print dataset as string
dataset_string = updated_df.to_string()
dataset_string

In [ ]:
    # Data Cleaning
    
    # Convert everything to lowercase
    dataset_string = dataset_string.lower() 
    
    # Remove mentions   
    dataset_string = re.sub('@','', dataset_string)  
    
    # Remove urls
    #dataset_string = re.sub(r'https?:\/\/.*\/\w*', '', dataset_string)
    
    # Remove hashtags
    dataset_string = re.sub(r'#\w*', '', dataset_string)    
    
    # Remove numbers
    dataset_string = re.sub(r'\d+', '', dataset_string)  
    
    # Remove punctuation
    #dataset_string = re.sub(r"[;':@#?!\&/$]+ *", ' ', dataset_string)
    
    # Remove that funny diamond
    dataset_string = re.sub(r"U+FFFD ", ' ', dataset_string)

    # Remove 'rt' denoting start of new message
    dataset_string = re.sub(r"rt ", ' ', dataset_string)

    # Print updated string
    dataset_string

Tweet Tone Analyzer

In [7]:
# Initialize Tone Analyser Client

# Create authentication object
authenticator = IAMAuthenticator(nlu_apikey)

# Create tone_analyzer instance
natural_language_understanding = NaturalLanguageUnderstandingV1 (
    version="2017-09-21",
    authenticator=authenticator
)

# Set the service endpoint
natural_language_understanding.set_service_url(nlu_url)

In [8]:
# Emotional Tone Analysis
response = natural_language_understanding.analyze(
    text=dataset_string,
    features=Features(emotion=EmotionOptions(targets=['monkeypox', 'Monkeypox']))).get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 3,
    "text_characters": 28583,
    "features": 1
  },
  "language": "en",
  "emotion": {
    "targets": [
      {
        "text": "monkeypox",
        "emotion": {
          "sadness": 0.437739,
          "joy": 0.197893,
          "fear": 0.122957,
          "disgust": 0.044644,
          "anger": 0.035869
        }
      },
      {
        "text": "monkeypox",
        "emotion": {
          "sadness": 0.437739,
          "joy": 0.197893,
          "fear": 0.122957,
          "disgust": 0.044644,
          "anger": 0.035869
        }
      }
    ],
    "document": {
      "emotion": {
        "sadness": 0.33264,
        "joy": 0.269106,
        "fear": 0.139371,
        "disgust": 0.060776,
        "anger": 0.046585
      }
    }
  }
}


In [10]:
# Sentiment Analysis
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(dataset_string)

{'neg': 0.119, 'neu': 0.799, 'pos': 0.082, 'compound': -0.997}